This is a notebook if a certain entity that is being asked is present or not

# Importing Libraries

In [9]:
!pip install validate_email_address

  Preparing metadata (setup.py) ... done
  Created wheel for validate_email_address: filename=validate_email_address-1-py3-none-any.whl size=5744 sha256=b54b8f94ad2c2ee874dab8bae1fea5cae243921821e9ec7a33843831424f226c
  Stored in directory: /root/.cache/pip/wheels/3c/e9/1a/7a6cf37b3628531dd5a10b4fa59031f1d815e5015bc55a4ce0
Successfully built validate_email_address


In [13]:
!pip install email-validator


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 7.6 MB/s eta 0:00:00


In [17]:
!pip install phonenumbers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 12.1 MB/s eta 0:00:00


In [33]:
!pip install usaddress

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 10.5 MB/s eta 0:00:00


In [39]:
!pip install dateutils

In [40]:
import spacy
import re
from validate_email_address import validate_email
from email_validator import validate_email, EmailNotValidError
import phonenumbers
import usaddress
from dateutil.parser import parse

# Testing for name

In [2]:
nlp = spacy.load("en_core_web_sm")

# Sample user input
user_input = "My name is John Doe."

# Process the user input with spaCy
doc = nlp(user_input)

# Extract named entities of type 'PERSON' (names)
names = [ent.text for ent in doc.ents if ent.label_ == 'PERSON']

# Print extracted names
print(names)  # Output: ['John Doe']

['John Doe']


In [3]:
def ret_name(str):
  doc=nlp(str)
  names = [ent.text for ent in doc.ents if ent.label_ == 'PERSON']
  return names

# Testing for Email


In [15]:
def extract_valid_emails(text):
    # Split the input text into potential email addresses
    potential_emails = text.split()

    valid_emails = []
    for potential_email in potential_emails:
        try:
            valid = validate_email(potential_email)
            valid_emails.append(valid.email)
        except EmailNotValidError:
            pass  # Invalid email, continue to the next one

    return valid_emails

In [16]:
x=extract_valid_emails('My email ID is vageesh1120@gmail.com')
print(x)

['vageesh1120@gmail.com']


# Testing for Phone Number

In [27]:
def extract_valid_phone_numbers(text):
    valid_phone_numbers = []

    for potential_number in phonenumbers.PhoneNumberMatcher(text, None):
        try:
            valid_number = phonenumbers.format_number(potential_number.number, phonenumbers.PhoneNumberFormat.E164)
            valid_phone_numbers.append(valid_number)
        except phonenumbers.NumberFormatException:
            pass  # Invalid phone number, continue to the next one

    return valid_phone_numbers

In [30]:
user_input = "You can call me at +919518002014"
valid_phone_numbers = extract_valid_phone_numbers(user_input)
print(valid_phone_numbers)

['+919518002014']


# Testing for Address

In [35]:
def extract_addresses(text):
    parsed_addresses = usaddress.parse(text)

    addresses = []
    current_address = {}
    for token, tag in parsed_addresses:
        if tag == 'AddressNumber':
            current_address['Number'] = token
        elif tag == 'StreetName':
            current_address['Street'] = token
        elif tag == 'StreetNamePostType':
            current_address['StreetType'] = token
        elif tag == 'City':
            current_address['City'] = token
        elif tag == 'StateName':
            current_address['State'] = token
        elif tag == 'ZipCode':
            current_address['Zip'] = token
            addresses.append(current_address)
            current_address = {}

    return addresses

In [38]:
user_input = "I live at 123 Main St, Springfield, IL 12345."

# Call the function to extract Indian address parts
extracted_address = extract_addresses(user_input)

# Print extracted address parts
print(extracted_address)

[{'Number': '123', 'Street': 'Main', 'StreetType': 'St,', 'State': 'IL', 'Zip': '12345.'}]


# Testing for Date of Birth

In [41]:
def extract_dob(text):
    dob = None
    try:
        dob = parse(text, fuzzy=True)
    except ValueError:
        pass  # Date not found or couldn't be parsed

    return dob

In [42]:
user_input = "My date of birth is January 15, 1990."

# Call the function to extract date of birth
extracted_dob = extract_dob(user_input)

# Print extracted date of birth
print(extracted_dob)

1990-01-15 00:00:00


# Testing for Education

In [47]:
def extract_education(text):
    doc = nlp(text)

    education_keywords = ["education", "degree", "qualification", "graduated", "studied", "university", "college", "school"]

    extracted_education = []
    education_info = ""
    for token in doc:
        if token.lower_ in education_keywords:
            start = token.i
            end = token.i + 10  # Maximum number of tokens to look ahead for education info
            for sub_token in doc[start:end]:
                if sub_token.ent_type_ == "ORG":  # Organization entity type (university name)
                    education_info = sub_token.text
                    break

    return education_info

In [49]:
user_input = "I have a Bachelor's degree in Computer Science from XYZ University."

# Call the function to extract education information
extracted_education = extract_education(user_input)

# Print extracted education information
print(extracted_education)

University
